In [1]:
import torch
import optuna
import optuna.visualization as vis
from torch.utils.tensorboard import SummaryWriter

from utils.metrices import roc_auc
from utils.read_data import IMDB_mlh
from MARA.MARA import MARA

from config import config

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Working on device: ", device)

imdb = IMDB_mlh().to(device)
imdb.info()

train_mask, val_mask, test_mask = imdb.get_training_mask(train_mask_size=0.25, val_mask_size=0.25)

writer = SummaryWriter(log_dir="./tensorboard_logs")

/home/lewy700/Documents/MARA/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-04 16:04:56.676778: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 16:04:57.051581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733324697.281361    7076 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733324697.326163    7076 cuda_blas.cc:1

MARA(
  (conv1): GCNConv(1000, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 52)
  (classifier): Linear(in_features=52, out_features=3, bias=True)
  (ReLU): ReLU6()
  (dropout): Dropout(p=0.001, inplace=False)
  (dropedge): DropEdge()
  (neuralsparse_1): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4001, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (neuralsparse_2): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4001, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (neuralsparse_3): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4001, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)
Working on device:  cuda
IMDB movie type dataset:
 Number of nodes: 5614
 Number of edges: 14715
 Number of edges: 

In [2]:
def train(data, model, criterion, optimizer, epoch, trial_number):
    model.train()
    optimizer.zero_grad()

    edges = torch.cat([data.layer_1, data.layer_2, data.cross_edges], dim=0).t()
    layers_lengths = torch.tensor([data.layer_1.shape[0], data.layer_2.shape[0], data.cross_edges.shape[0]], dtype=torch.int64)

    out, _, _ = model(data.node_features, edges, layers_lengths)

    train_loss = criterion(out[train_mask], data.classes[train_mask])
    train_loss.backward()
    optimizer.step()

    train_score = roc_auc(out[train_mask], data.classes[train_mask])
    val_score = roc_auc(out[val_mask], data.classes[val_mask])

    writer.add_scalar(f"Trial {trial_number}/Loss/train", train_loss.item(), epoch)
    writer.add_scalar(f"Trial {trial_number}/ROC-AUC/train", train_score, epoch)
    writer.add_scalar(f"Trial {trial_number}/ROC-AUC/val", val_score, epoch)

    return train_loss.item(), train_score, val_score

In [9]:
def objective(trial):
    params = {
        "simplification_type": "l-b-l",
        "simplification_stages": "each",
        "DE_p": trial.suggest_float("DE_p", 0, 1),
        "lr": trial.suggest_float("lr", 1e-5, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
        "dropout": trial.suggest_float("dropout", 0, 0.7)
    }

    mara = MARA(
        simplification_type=params["simplification_type"],
        simplification_stages=params["simplification_stages"],
        DE_p=params["DE_p"],
        dropout=params["dropout"]
    ).to(device)
    crit = torch.nn.CrossEntropyLoss()
    optim = torch.optim.Adam(mara.parameters(), lr=params["lr"], weight_decay=params["weight_decay"])

    early_stopping = {
        "best_val_score": 0,
        "patience": 15,
        "counter": 0,
        "best_weights": None
    }

    for epoch in range(251):
        train_loss, train_score, val_score = train(imdb, mara, crit, optim, epoch, trial.number)

        if val_score > early_stopping["best_val_score"]:
            early_stopping = {
                "best_val_score": val_score,
                "counter": 0,
                "best_weights": mara.state_dict(),
                "patience": early_stopping["patience"]
            }
        else:
            early_stopping["counter"] += 1

        if early_stopping["counter"] >= early_stopping["patience"]:
            break

    mara.load_state_dict(early_stopping["best_weights"])

    trial_number = trial.number
    writer.add_scalar(f"Trial {trial_number}/Best ROC-AUC/val", early_stopping["best_val_score"], trial_number)
    for param_name, param_value in params.items():
        writer.add_text(f"Trial {trial_number}/Params", f"{param_name}: {param_value}", trial_number)

    return early_stopping["best_val_score"]

In [4]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)

[I 2024-12-01 22:07:44,849] A new study created in memory with name: no-name-619c9c64-289f-472d-af21-507bf8c10f5f


[I 2024-12-01 22:07:53,934] Trial 0 finished with value: 0.6209567904717967 and parameters: {'DE_p': 0.9572910630287309, 'lr': 4.343413588651947e-05, 'weight_decay': 6.114680830252111e-05, 'dropout': 0.32009144331442385}. Best is trial 0 with value: 0.6209567904717967.
[I 2024-12-01 22:07:59,235] Trial 1 finished with value: 0.5946739145297784 and parameters: {'DE_p': 0.10816274040943652, 'lr': 1.4868215597240196e-05, 'weight_decay': 0.00015343177083790402, 'dropout': 0.43197723956903106}. Best is trial 0 with value: 0.6209567904717967.
[I 2024-12-01 22:08:04,845] Trial 2 finished with value: 0.5970818246455877 and parameters: {'DE_p': 0.37644824163160306, 'lr': 0.00011248811631164244, 'weight_decay': 0.000880288717168461, 'dropout': 0.4096913084745622}. Best is trial 0 with value: 0.6209567904717967.
[I 2024-12-01 22:08:09,987] Trial 3 finished with value: 0.6667528334337116 and parameters: {'DE_p': 0.5662582579638341, 'lr': 7.648771070656797e-05, 'weight_decay': 3.7772536482636503e-0

In [5]:
best_trial = study.best_trial

print("Best trial:")
print(f"  Value: {best_trial.value}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value: 0.8775028626160709
  Params: 
    DE_p: 0.9812426647949694
    lr: 0.001039111537208772
    weight_decay: 0.00020220373700746978
    dropout: 0.05922335978666936


In [6]:
vis.plot_slice(study)

In [7]:
vis.plot_optimization_history(study)

In [8]:
vis.plot_param_importances(study)